# PythonとKerasによるディープラーニング 第5章 CNN

# データ

ここではデータセットのダウンロードと前処理を行います

データセット: Fashion-MNIST  
説明: 衣類品の画像のデータセット。10カテゴリーに分かれている。

ラベル カテゴリー   
0. T-シャツ/トップ (T-shirt/top)  
1. ズボン (Trouser)  
2. プルオーバー (Pullover)  
3. ドレス (Dress)  
4. コート (Coat)  
5. サンダル (Sandal)  
6. シャツ (Shirt)  
7. スニーカー (Sneaker)  
8. バッグ (Bag)  
9. アンクルブーツ (Ankle boot)  

In [ ]:
# クラス名(ラベルデータは0〜9までの整数．class_namesはそれに対応した名前)
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# データのダウンロード
from tensorflow.keras.datasets import fashion_mnist
(_X_train_full, _y_train_full), (_X_test, _y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step



変数の前に"_"がついるのは前処理を行う前のデータとして残すため  
それと複数回セルを実行してしまったときに変なことが起こらないようにするためでもある

In [ ]:
# データの確認
# X:(sample, width, height), y:(sample)
_X_train_full.shape, _X_train_full.dtype, _y_train_full.shape, _y_train_full.dtype

((60000, 28, 28), dtype('uint8'), (60000,), dtype('uint8'))

In [ ]:
# fashion mnistの画像を表示してみる
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
for i in range(10):
    data = [(x, t) for x, t in zip(_X_train_full, _y_train_full) if t == i]
    # カテゴリーiの中でサンプルを一つ取ってくる
    x, y = data[0]
    plt.subplot(2, 5, i+1)
    # タイトルの表示
    plt.title("{}: {}\n{} pic".format(i, class_names[i], len(data)))
    # 座標軸を非表示にする
    plt.axis("off")
    # 画像表示
    plt.imshow(x, cmap="binary")

In [ ]:
# trainデータとvalidationデータに分ける
from sklearn.model_selection import train_test_split
_X_train, _X_val, _y_train, _y_val = train_test_split(_X_train_full, _y_train_full, test_size=0.1, random_state=1)

In [ ]:
import numpy as np

In [ ]:
# 前処理
# レンジを0-1にするために255で割る
X_train = _X_train.astype("float32") / 255
X_val = _X_val.astype("float32") / 255
X_test = _X_test.astype("float32") / 255
# kerasの入力データの形式は(ミニバッチサイズ、横幅、縦幅、チャネル数)
# (sample, width, height) -> (sample, width, height, channel)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]
"""
(smple, width, height) -> (sample, width, height, channel)の操作は
X_train = X_train.reshape(-1, 28, 28, 1)
でもできる
"""
y_train = _y_train
y_val = _y_val
y_test = _y_test
# 確認
X_train.shape, X_train.dtype, y_train.shape, y_train.dtype

((54000, 28, 28, 1), dtype('float32'), (54000,), dtype('uint8'))

# モデル

ここでは、CNNのモデルを作ります


モデルの作り方はおおまかに以下の3通りくらいあります  
- シーケンシャルAPI
  - 各層が1入力1出力のシンプルなモデルを作る時に使えます
- 関数型API
  - 複数入力、複数出力、一部重み共有など複雑なモデルに対応できます
- サブクラス化API
  - モデル内にループや条件分岐などのダイナミックな動作が含まれるモデルに対応できます
  - 基本的にはシーケンシャルAPIや関数型APIで対応できない場合のみ利用する

今回はシーケンシャルAPIを使います

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
# モデル定義
model = Sequential()
model.add(Conv2D(64, 7, activation="relu", padding="same", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(2))
model.add(Conv2D(128, 3, activation="relu", padding="same"))
model.add(Conv2D(128, 3, activation="relu", padding="same"))
model.add(MaxPooling2D(2))
model.add(Conv2D(256, 3, activation="relu", padding="same"))
model.add(Conv2D(256, 3, activation="relu", padding="same"))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
# 図でもモデルを見てみる
from tensorflow.keras.utils import plot_model
plot_model(model)
# plot_model(model, to_file="file_path")みたいにするとfileにも書き込める

In [ ]:
#@title optimizerの設定
optimizer_name = "nadam" #@param ["rmsprop", "adam", "nadam"]


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer_name,
              metrics=["accuracy"])

lossで損失関数を指定  
sparce_categorical_crossentropyは多クラス分類でかつラベルが整数のときに使う  
ラベルがone-hotの状態になっているときはcategorical_crossentropyを選ぶ

optimizerは最適化するときに使うアルゴリズムを指定  
有名なのは大体使える、詳細はkeras公式サイトで

metricsはモデルの性能を図る指標を指定

# 学習と評価

In [ ]:
#@title ハイパーパラメータの設定 { run: "auto"}
batch_size =  128#@param {type:"integer"}
epochs =  10#@param {type:"integer"}

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val))

In [ ]:
# 確認
X_test.shape, X_test.dtype, y_test.shape, y_test.dtype

In [ ]:
# testデータで評価
score = model.evaluate(X_test, y_test)

In [ ]:
# accuracyとlossをグラフで見る
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1) # 縦の範囲を0から1までに
plt.show()

In [ ]:
# 実際にモデルの予測結果を見る
X_new = X_test[:10]
y_pred = model.predict(X_new)
y_pred.round(2)

In [ ]:
# こうすると推定確率が最も高いクラスのみを取ってこれる
np.argmax(model.predict(X_new), axis=-1)

In [ ]:
# 画像と予測結果
X_new = X_test[:20]
y_pred = np.argmax(model.predict(X_new), axis=-1)
plt.figure(figsize=(10, 10))
for i in range(20):
    plt.subplot(4, 5, i+1)
    color = "green" if y_pred[i] == y_test[i] else "red" # 正解は緑、不正解は赤で表示
    plt.title("pred: {}\ngold: {}".format(class_names[y_pred[i]], class_names[y_test[i]]), color=color)
    plt.axis("off")
    plt.imshow(X_new[i].reshape(28, 28), cmap="binary")